In [3]:
from gurobipy import GRB, Model, quicksum
import gurobipy as gb
import pandas as pd

In [4]:
food_categories_df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/food_categories.csv')
food_preferences_df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/food_preferences2.csv')
nutrient_requirements_df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/nutrient_requirements.csv')
nutrient_content_df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/nutrient_content.csv')

In [17]:
# Create the model
model = gb.Model("Optimum Diet")

# Food items/categories
food_items = food_categories_df['Food_Item'].tolist()
cost_per_gram = food_categories_df['Cost_per_gram'].tolist()

# Nutrient requirements
names_nutrients = nutrient_requirements_df['Nutrient'].tolist()
min_requirements = nutrient_requirements_df['Min_Requirement'].tolist()
max_requirements = nutrient_requirements_df['Max_Requirement'].tolist()

# Hardcoded dietary preference requirements
diet_total = {
    'Vegetarian': 46160,
    'Vegan': 11540,
    'Kosher': 17310,
    'Halal': 92320,
    'All': 577000
}

# Hardcoded dietary preferences
diet_preferences = ['Vegetarian', 'Vegan', 'Halal', 'Kosher', 'All']

# Decision variables
food_quantities = model.addVars(food_items, lb=0, vtype=GRB.CONTINUOUS, name="Food")
total_diet_quantity = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="TotalDietQuantity")

# Dietary preference constraints
for preference in diet_preferences:
    if preference == 'All':
        # Sum of quantities of all food items should not exceed the total for 'All' preference
        model.addConstr(gb.quicksum(food_quantities[food] for food in food_items) <= diet_total[preference], f"Total_{preference}")
    else:
        # Check if the preference column exists in the food categories DataFrame
        if f'Is_{preference}' in food_categories_df.columns:
            # Create a filtered list of food items for each specific dietary preference
            food_list = food_categories_df[food_categories_df[f'Is_{preference}'] == True]['Food_Item'].tolist()  # Extract food items
            # Sum of quantities of food items in the specific dietary list should not exceed the total for that preference
            model.addConstr(gb.quicksum(food_quantities[food] for food in food_list) <= diet_total[preference], f"Total_{preference}")
        else:
            print(f"Error: 'Is_{preference}' column not found in food categories DataFrame.")

# Nutritional balance constraints
for i, nutrient in enumerate(names_nutrients):
    model.addConstr(gb.quicksum(nutrient_content_df.loc[j, nutrient] * food_quantities[food] for j, food in enumerate(food_items)) >= min_requirements[i], f"Min_{nutrient}")
    model.addConstr(gb.quicksum(nutrient_content_df.loc[j, nutrient] * food_quantities[food] for j, food in enumerate(food_items)) <= max_requirements[i], f"Max_{nutrient}")

# Variety constraint
for food in food_items:
    model.addConstr(food_quantities[food] <= 0.03 * total_diet_quantity, f"Variety_{food}")

# Objective function: Minimize total cost
model.setObjective(gb.quicksum(cost_per_gram[i] * food_quantities[food] for i, food in enumerate(food_items)), GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Print the solution
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for food in food_items:
        quantity = food_quantities[food].X
        if quantity > 0:
            print(f"{food}: {quantity} grams")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 245 rows, 121 columns and 14928 nonzeros
Model fingerprint: 0x7a79486f
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+05]
Presolve removed 182 rows and 1 columns
Presolve time: 0.01s
Presolved: 63 rows, 180 columns, 7464 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.370451e+04   0.000000e+00      0s

Solved in 43 iterations and 0.02 seconds (0.01 work units)
Infeasible model
No optimal solution found.
